# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
recordCount, setCount = 0, 0

url = "http://api.openweathermap.org/data/2.5/weather?APPID=" + weather_api_key 
print('------------------------')
print('Beginning Data Retrieval')
print('------------------------')

for city in cities:
    query = f"{url}&q={city}"
    response = requests.get(query).json()

    if recordCount < 50:
        recordCount += 1
    else:
        setCount += 1
        recordCount = 0
    
    print(f"Processing record {recordCount} of set {setCount} | {city}")

print('-------------------------')
print('Data Retrieval Complete')
print('-------------------------')

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?APPID=" + weather_api_key 

new_city = []
lat = []
lng = []
temp = []
humidity = []
cloudiness = []
wind = []
country = []
date = []

for city in cities:
    query = f"{url}&q={city}"
    response = requests.get(query).json()
    
    try:
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        temp.append(response['main']['temp_max'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        wind.append(response['wind']['speed'])
        new_city.append(city)
    except:
        pass

weather_dict = {
    "City": new_city,
    "Lat": lat,
    "Lng": lng,
    "MaxTemp": temp,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind,
    "Country": country,
    "Date": date,
}

weather_data = pd.DataFrame(weather_dict)
weather_data.to_csv(output_data_file)
weather_data.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
weather_data[weather_data['Humidity'] > 100]

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
import datetime
date = datetime.datetime.fromtimestamp(int(weather_data['Date'][0])).strftime('%m/%d/%Y')

In [ ]:
weather_data.plot(kind='scatter', x='Lat', y='MaxTemp', c='DarkBlue')
plt.title(f'City Latitude Vs Max Temperature ({date})')
plt.xlabel('Latitude')
plt.ylabel('Max temperature (F)')
plt.grid()
plt.show()
plt.savefig("../Images/LatitudeVsTemperature.png")

## Latitude vs. Humidity Plot

In [ ]:
weather_data.plot(kind='scatter', x='Lat', y='Humidity', c='DarkBlue')
plt.title(f'City Latitude Vs Humidity ({date})')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.grid()
plt.savefig("../Images/LatitudeVsHumidity.png")

## Latitude vs. Cloudiness Plot

In [ ]:
weather_data.plot(kind='scatter', x='Lat', y='Cloudiness', c='DarkBlue')
plt.title(f'City Latitude Vs Cloudiness ({date})')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.grid()
plt.savefig("../Images/LatitudeVsCloudiness.png")

## Latitude vs. Wind Speed Plot

In [ ]:
weather_data.plot(kind='scatter', x='Lat', y='Wind Speed', c='DarkBlue')
plt.title(f'City Latitude Vs Wind Speed ({date})')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.grid()
plt.savefig("../Images/LatitudeVsWindSpeed.png")

## Linear Regression

In [ ]:
# Divide Northern Hemisphere and Southern Hemisphere
northern_hemisphere = weather_data[weather_data['Lat'] >= 0]
southern_hemisphere = weather_data[weather_data['Lat'] < 0]

In [ ]:
def makeLinRegression(df, x_Col, y_Col):
    (regr_slope, regr_intercept, regr_r_val, _, _) = linregress(df[x_Col], df[y_Col])
    regr_line_x = np.array([a for a in range(int(min(df[x_Col].values)), int(max(df[x_Col].values)+1))])
    regr_line_y = regr_slope * regr_line_x + regr_intercept
    line_eq = "y = " + str(round(regr_slope,2)) + "x + " + str(round(regr_intercept,2))

    df_axes = df.plot(kind="scatter", x=x_Col, y=y_Col, 
                    title=f"{y_Col} vs. {x_Col} ({date})")

    df_axes.plot(regr_line_x, regr_line_y, 'r')
    
    plt.annotate(line_eq, (min(regr_line_x)+10, min(regr_line_y)+10), fontsize=12, color="red")

    print(f"The r-value is: {regr_r_val:.2f}")
                  
    return df_axes

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
n_temp_lat_axes = makeLinRegression(northern_hemisphere, "Lat", "MaxTemp")
n_temp_lat_axes.get_figure().savefig("../Images/MaxTempVSLatitudeNorthernHemisphere.png")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
s_temp_lat_axes = makeLinRegression(southern_hemisphere, "Lat", "MaxTemp")
s_temp_lat_axes.get_figure().savefig("../Images/MaxTempVSLatitudeSouthernHemisphere.png")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
n_humidity_lat_axes = makeLinRegression(northern_hemisphere, "Lat", "Humidity")
n_humidity_lat_axes.get_figure().savefig("../Images/HumidityVSLatitudeNorthernHemisphere.png")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
s_humidity_lat_axes = makeLinRegression(southern_hemisphere, "Lat", "Humidity")
s_humidity_lat_axes.get_figure().savefig("../Images/HumidityvSLatitudeSouthernHemisphere.png")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
n_cloudiness_lat_axes = makeLinRegression(northern_hemisphere, "Lat", "Cloudiness")
n_cloudiness_lat_axes.get_figure().savefig("../Images/CloudinessVSLatitudeNorthernHemisphere.png")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
s_cloudiness_lat_axes = makeLinRegression(southern_hemisphere, "Lat", "Cloudiness")
s_cloudiness_lat_axes.get_figure().savefig("../Images/CloudinessVSLatitudeSouthernHemisphere.png")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
n_wind_lat_axes = makeLinRegression(northern_hemisphere, "Lat", "Wind Speed")
n_wind_lat_axes.get_figure().savefig("../Images/WindSpeedVSLatitudeNorthernHemisphere.png")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
s_wind_lat_axes=makeLinRegression(southern_hemisphere, "Lat", "Wind Speed")
s_wind_lat_axes.get_figure().savefig("../Images/WindSpeedVSLatitudeSouthernHemisphere.png")